# Derivation

In [1]:
import sympy as sp
import numpy as np

def norm(vec: sp.Matrix):
    return vec.dot(vec)

In [2]:
t = sp.symbols('t')
x = sp.Function('x')(t)
y = sp.Function('y')(t)
theta = sp.Function('theta')(t)
l, m_p, m_c, I_p, g = sp.symbols('l, m_p, m_c, I_p, g')

In [3]:
sym_x = sp.symbols(r'x')
sym_x_d = sp.symbols(r'\dot{x}')
sym_x_d2 = sp.symbols(r'\ddot{x}')
sym_y = sp.symbols(r'y')
sym_y_d = sp.symbols(r'\dot{y}')
sym_y_d2 = sp.symbols(r'\ddot{y}')
sym_theta = sp.symbols(r'\theta')
sym_theta_d = sp.symbols(r'\dot{\theta}')
sym_theta_d2 = sp.symbols(r'\ddot{\theta}')
alias = {
    x: sym_x,
    x.diff(t): sym_x_d,
    x.diff(t).diff(t): sym_x_d2,
    y: sym_y,
    y.diff(t): sym_y_d,
    y.diff(t).diff(t): sym_y_d2,
    theta: sym_theta,
    theta.diff(t): sym_theta_d,
    theta.diff(t).diff(t): sym_theta_d2,
}

In [4]:
C = sp.Matrix([x, 0])
C.subs(alias)

Matrix([
[x],
[0]])

In [5]:
P = sp.Matrix([x + l * sp.sin(theta), - l * sp.cos(theta)])
P.subs(alias)

Matrix([
[l*sin(\theta) + x],
[   -l*cos(\theta)]])

In [6]:
T_c = m_c * norm(C.diff(t)) / 2
T_c.subs(alias)

\dot{x}**2*m_c/2

In [7]:
T_p = (m_p * norm(P.diff(t)) / 2 + I_p * (theta.diff(t)**2) / 2).simplify()
T_p.subs(alias)

I_p*\dot{\theta}**2/2 + m_p*(\dot{\theta}**2*l**2 + 2*\dot{\theta}*\dot{x}*l*cos(\theta) + \dot{x}**2)/2

In [8]:
U_p = -m_p * g * l * sp.cos(theta)
U_p.subs(alias)

-g*l*m_p*cos(\theta)

In [9]:
L = (T_c + T_p - U_p).simplify()
L.subs(alias)

I_p*\dot{\theta}**2/2 + \dot{x}**2*m_c/2 + g*l*m_p*cos(\theta) + m_p*(\dot{\theta}**2*l**2 + 2*\dot{\theta}*\dot{x}*l*cos(\theta) + \dot{x}**2)/2

In [10]:
L_by_theta = (L.diff(theta.diff(t)).diff(t) - L.diff(theta)).simplify().subs(alias)
L_by_theta

I_p*\ddot{\theta} + \ddot{\theta}*l**2*m_p + \ddot{x}*l*m_p*cos(\theta) + g*l*m_p*sin(\theta)

In [11]:
L_by_x = (L.diff(x.diff(t)).diff(t) - L.diff(x)).simplify().subs(alias)
L_by_x

\ddot{x}*m_c + m_p*(\ddot{\theta}*l*cos(\theta) + \ddot{x} - \dot{\theta}**2*l*sin(\theta))

In [12]:
sym_theta_d2_by = sp.solve(L_by_theta, sym_theta_d2)[0]
sym_theta_d2_by

-l*m_p*(\ddot{x}*cos(\theta) + g*sin(\theta))/(I_p + l**2*m_p)

In [13]:
dynamic_expression = L_by_x.subs({sym_theta_d2: sym_theta_d2_by})
dynamic_expression

\ddot{x}*m_c + m_p*(\ddot{x} - \dot{\theta}**2*l*sin(\theta) - l**2*m_p*(\ddot{x}*cos(\theta) + g*sin(\theta))*cos(\theta)/(I_p + l**2*m_p))

# Сonsiderations about maximization

Let's transform the expression

In [14]:
dynamic_expression.expand().collect(sym_x_d2*m_p).collect(sym_x_d2)

\ddot{x}*(m_c + m_p*(-l**2*m_p*cos(\theta)**2/(I_p + l**2*m_p) + 1)) - \dot{\theta}**2*l*m_p*sin(\theta) - g*l**2*m_p**2*sin(\theta)*cos(\theta)/(I_p + l**2*m_p)

Note that the multiplier $\ddot{x}$ is always positive. Further, without detracting from generality, we will assume that $\ddot{x} > 0$. Note that replacing $\theta \to -\theta'$ changes the sign of the last two terms, while replacing $\theta \to \theta' + \pi$ changes the sign of the second summand. This means that the maximum is reached in the quarter $[-\frac{\pi}{2}, 0]$, because the summands will have a plus sign. Then to maximize all sums, we also have to substitute the maximum $\ddot{x}$ and $\dot{\theta}$.

# Function

In [15]:
def max_cart_linear_force(
    *,
    cart_mass,                      # kg
    pendulum_mass,                  # kg
    pendulum_moment_of_inertia,     # kg * m^2
    distance_between_centers,       # m
    max_pendulum_angular_velocity,  # rad / s
    max_cart_acceleration,          # m / s^2
) -> float:                         # N ((kg * m) / s^2)
    ga = 9.8                        # earth gravitational acceleration
    func_by_sym_theta = dynamic_expression.subs(
        {   
            g: ga,
            m_c: cart_mass,
            m_p: pendulum_mass,
            I_p: pendulum_moment_of_inertia,
            l: distance_between_centers,
            sym_theta_d: max_pendulum_angular_velocity,
            sym_x_d2: max_cart_acceleration,
        }
    )
    optimized_func = sp.lambdify(sym_theta, func_by_sym_theta)
    return max(optimized_func(np.linspace(0, 2*np.pi, 2000)))

In [16]:
max_pendulum_angular_velocity = 3*(2*np.pi)
max_cart_acceleration = 5

cart_mass=0.1
pendulum_mass=0.118
pendulum_length=0.3

pendulum_moment_of_inertia = (1/12) * pendulum_mass * pendulum_length**2
distance_between_centers = pendulum_length / 2

In [17]:
# example
max_cart_linear_force(
    cart_mass=cart_mass,
    pendulum_mass=pendulum_mass,
    pendulum_moment_of_inertia=pendulum_moment_of_inertia,
    distance_between_centers=distance_between_centers,
    max_pendulum_angular_velocity=max_pendulum_angular_velocity,
    max_cart_acceleration=max_cart_acceleration,
)

7.430438529490409